In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:

spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Get:5 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Get:7 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:8 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Fetched 222 kB in 3s (86.5 kB/s)
Reading package lists... Done


In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Import sql functions
from pyspark.sql.functions import year, avg, round

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

#Add a column of year to dataframe
df = df.withColumn("year", year(df['date']))

df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|year|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|2022|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|2021|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|2019|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|2019|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|2022|
|5aa00529-0533-46b...|2019-01-30|      2

In [ ]:
# 2. Create a temporary view of the DataFrame.

df.createOrReplaceTempView('sales')

In [ ]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

spark.sql("SELECT year, round(avg(price),2) as `Average 4-bedroom Sale Price` FROM sales WHERE bedrooms=4 GROUP BY year").show()


+----+----------------------------+
|year|Average 4-bedroom Sale Price|
+----+----------------------------+
|2022|                   296363.88|
|2019|                    300263.7|
|2020|                   298353.78|
|2021|                   301819.44|
+----+----------------------------+



In [ ]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

spark.sql("SELECT date_built as Year, round(avg(price),2) as `Average 3-bed, 3-bath Sale Price` FROM sales WHERE bedrooms=3 and bathrooms=3 GROUP BY date_built").show()



+----+--------------------------------+
|Year|Average 3-bed, 3-bath Sale Price|
+----+--------------------------------+
|2016|                       290555.07|
|2012|                       293683.19|
|2017|                       292676.79|
|2014|                       290852.27|
|2013|                       295962.27|
|2011|                       291117.47|
|2015|                        288770.3|
|2010|                       292859.62|
+----+--------------------------------+



In [ ]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

spark.sql("SELECT date_built as Year, round(avg(price),2) as `Average Sale Price` FROM sales WHERE bedrooms=3 and bathrooms=3 and floors=2 and sqft_living>2000 GROUP BY date_built").show()


+----+------------------+
|Year|Average Sale Price|
+----+------------------+
|2016|          293965.1|
|2012|         307539.97|
|2017|         280527.62|
|2014|         297619.46|
|2013|         303676.79|
|2011|         276553.81|
|2015|         297609.97|
|2010|         285010.22|
+----+------------------+



In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()


spark.sql("SELECT round(avg(view),2) as `Average View Rating`, round(avg(price),2) as `Average Sale Price` FROM sales WHERE price>=350000 GROUP BY date_built").show()


print("--- %s seconds ---" % (time.time() - start_time))

+-------------------+------------------+
|Average View Rating|Average Sale Price|
+-------------------+------------------+
|              32.91|         478550.09|
|              33.48|         475524.98|
|              32.03|         475884.72|
|              32.28|         475674.82|
|              32.61|         476837.88|
|              32.28|         472861.91|
|              31.89|         471517.37|
|              30.64|         463301.08|
+-------------------+------------------+

--- 0.6753075122833252 seconds ---


In [ ]:
# 7. Cache the the temporary table sales.
spark.sql("cache table sales")

DataFrame[]

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('sales')

True

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

spark.sql("SELECT round(avg(view),2) as `Average View Rating`, round(avg(price),2) as `Average Sale Price` FROM sales WHERE price>=350000 GROUP BY date_built").show()

print("--- %s seconds ---" % (time.time() - start_time))


+-------------------+------------------+
|Average View Rating|Average Sale Price|
+-------------------+------------------+
|              32.91|         478550.09|
|              33.48|         475524.98|
|              32.03|         475884.72|
|              32.28|         475674.82|
|              32.61|         476837.88|
|              32.28|         472861.91|
|              31.89|         471517.37|
|              30.64|         463301.08|
+-------------------+------------------+

--- 0.34838294982910156 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy("date_built").mode("overwrite").parquet("sales_partitioned")

In [ ]:
# 11. Read the parquet formatted data.
p_df=spark.read.parquet('sales_partitioned')

In [ ]:
# 12. Create a temporary table for the parquet data.
p_df.createOrReplaceTempView('sales_partitioned')

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("SELECT round(avg(view),2) as `Average View Rating`, round(avg(price),2) as `Average Sale Price` FROM sales_partitioned WHERE price>=350000").show()


print("--- %s seconds ---" % (time.time() - start_time))

+-------------------+------------------+
|Average View Rating|Average Sale Price|
+-------------------+------------------+
|              32.26|         473796.26|
+-------------------+------------------+

--- 0.9732301235198975 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table sales")


DataFrame[]

In [ ]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached("sales"):
  print("a table is till cached")
else:
  print("all clear")


all clear
